In [2]:
:dep graph-ds = { path = "." }

In [ ]:
use graph_ds::{Graph, Node, Edge};
use graph_ds::hexagon_graph::{hexagon_graph_from_file, cell::Cell};

In [ ]:
let mut graph : Graph<Cell> = hexagon_graph_from_file("de_inferno_hexagons.mpk")?;

In [ ]:
graph.edges.len()

In [ ]:
let start = graph.nodes.get(0).unwrap().as_ref().unwrap().id;
let end = graph.nodes.get(4).unwrap().as_ref().unwrap().id;

In [ ]:
graph.bfs(start, Some(end))